# Animation
## Import Modules

In [ ]:
import numpy as np
from sectionproperties.pre.geometry import Geometry
from shapely import Polygon

import concreteproperties.stress_strain_profile as ssp
from concreteproperties.concrete_section import ConcreteSection
from concreteproperties.material import Concrete, SteelBar
from concreteproperties.pre import add_bar_rectangular_array

## Define Materials

In [ ]:
concrete = Concrete(
    name="40 MPa Concrete",
    density=2.4e-6,
    stress_strain_profile=ssp.EurocodeNonLinear(
        elastic_modulus=32.8e3,
        ultimate_strain=0.0035,
        compressive_strength=40,
        compressive_strain=0.0023,
        tensile_strength=3.8,
        tension_softening_stiffness=10e3,
    ),
    ultimate_stress_strain_profile=ssp.RectangularStressBlock(
        compressive_strength=40,
        alpha=0.79,
        gamma=0.87,
        ultimate_strain=0.003,
    ),
    flexural_tensile_strength=3.8,
    colour="lightgrey",
)

steel = SteelBar(
    name="500 MPa Steel",
    density=7.85e-6,
    stress_strain_profile=ssp.SteelElasticPlastic(
        yield_strength=500,
        elastic_modulus=200e3,
        fracture_strain=0.05,
    ),
    colour="grey",
)

## Construct Geometry

In [ ]:
# construct box by subtracting an inner rectangle from an outer rectangle
outer_points = [
    [50, 0],
    [850, 0],
    [900, 50],
    [900, 1150],
    [850, 1200],
    [50, 1200],
    [0, 1150],
    [0, 50],
]
inner_points = [
    [50, 0],
    [550, 0],
    [600, 50],
    [600, 850],
    [550, 900],
    [50, 900],
    [0, 850],
    [0, 50],
]
outer = Geometry(geom=Polygon(outer_points), material=concrete)
inner = Geometry(geom=Polygon(inner_points)).align_center(align_to=outer)
geom = outer - inner

# add bottom bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=620,
    material=steel,
    n_x=9,
    x_s=750 / 8,
    anchor=(75, 75),
    n=16,
)

# add top bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=310,
    material=steel,
    n_x=9,
    x_s=750 / 8,
    anchor=(75, 1125),
    n=16,
)

# add side bars
geom = add_bar_rectangular_array(
    geometry=geom,
    area=200,
    material=steel,
    n_x=2,
    x_s=750,
    n_y=6,
    y_s=150,
    anchor=(75, 225),
    n=16,
)

conc_sec = ConcreteSection(geom)
conc_sec.plot_section()

## Perform Moment-Curvature Analysis

In [ ]:
mk_res = conc_sec.moment_curvature_analysis()

In [ ]:
mk_res.plot_results()

## Generate Stress Results

In [ ]:
kappa_list = np.arange(0, 2e-5, 5e-8)  # curvatures
n_kappa = len(kappa_list)
stress_res_list = []
moment_list = [0]

for idx, kappa in enumerate(kappa_list[1:]):
    stress_res = conc_sec.calculate_service_stress(
        moment_curvature_results=mk_res, m=None, kappa=kappa
    )
    stress_res_list.append(stress_res)
    moment_list.append(stress_res.sum_moments())

    # log progress
    if idx % 20 == 0:
        print(f"{idx/(n_kappa-1)*100:.2f}% complete")

print("100% complete")

## Create Plotting Function

In [ ]:
import warnings

import matplotlib.cm as cm
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import matplotlib.tri as tri
from matplotlib.collections import PatchCollection
from matplotlib.colors import CenteredNorm

from concreteproperties.post import plotting_context


def plot_stress_and_graph(res_idx, mk_res_list, k_list, m_list, **kwargs):
    """Function that plots the animation."""
    with plotting_context(title="Stress", **kwargs) as (fig, ax):
        # plot background
        mk_res_list[res_idx].concrete_section.plot_section(
            background=True, **dict(kwargs, ax=ax)
        )

        # set up the colormaps
        cmap_conc = cm.get_cmap(name="RdGy")
        cmap_steel = cm.get_cmap(name="bwr")

        # determine minimum and maximum stress values for the contour list
        # add tolerance for plotting stress blocks
        conc_sig_min = (
            min([min(x) for x in mk_res_list[res_idx].concrete_stresses]) - 1e-12
        )
        conc_sig_max = (
            max([max(x) for x in mk_res_list[res_idx].concrete_stresses]) + 1e-12
        )
        # steel_sig_min = min(mk_res_list[res_idx].lumped_reinforcement_stresses)
        # steel_sig_max = max(mk_res_list[res_idx].lumped_reinforcement_stresses)

        # set up ticks
        v_conc = np.linspace(conc_sig_min, conc_sig_max, 15, endpoint=True)
        # v_steel = np.linspace(steel_sig_min, steel_sig_max, 15, endpoint=True)
        # ticks_conc = v_conc
        # ticks_steel = v_steel

        # plot the concrete stresses
        for idx, sig in enumerate(mk_res_list[res_idx].concrete_stresses):
            # check region has a force
            if abs(mk_res_list[res_idx].concrete_forces[idx][0]) > 1e-8:
                # create triangulation
                triang = tri.Triangulation(
                    mk_res_list[res_idx]
                    .concrete_analysis_sections[idx]
                    .mesh_nodes[:, 0],
                    mk_res_list[res_idx]
                    .concrete_analysis_sections[idx]
                    .mesh_nodes[:, 1],
                    mk_res_list[res_idx]
                    .concrete_analysis_sections[idx]
                    .mesh_elements[:, 0:3],
                )

                # plot the filled contour
                ax.tricontourf(triang, sig, v_conc, cmap=cmap_conc, norm=CenteredNorm())

                # plot a zero stress contour, supressing warning
                with warnings.catch_warnings():
                    warnings.filterwarnings(
                        "ignore",
                        message="No contour levels were found within the data range.",
                    )

                    # set zero stress for neutral axis contour
                    zero_level = 0

                    if min(sig) > 0 and min(sig) < 1e-3:
                        zero_level = min(sig) + 1e-12

                    if max(sig) < 0 and max(sig) > -1e-3:
                        zero_level = max(sig) - 1e-12

                    if min(sig) == 0:
                        zero_level = 1e-12

                    if max(sig) == 0:
                        zero_level = -1e-12

                    ax.tricontour(
                        triang, sig, [zero_level], linewidths=1, linestyles="dashed"
                    )

        # plot the steel stresses
        steel_patches = []
        colours = []

        for idx, sig in enumerate(mk_res_list[res_idx].lumped_reinforcement_stresses):
            steel_patches.append(
                mpatches.Polygon(
                    xy=list(
                        mk_res_list[res_idx]
                        .concrete_section.reinf_geometries_lumped[idx]
                        .geom.exterior.coords
                    )
                )
            )
            colours.append(sig)

        patch = PatchCollection(steel_patches, cmap=cmap_steel)
        patch.set_array(colours)
        ax.add_collection(patch)
        ax.set_aspect("equal", anchor="C")
        ax.set_xlim(0, 900)
        ax.set_ylim(0, 1200)

        fig = ax.get_figure()
        fig.axes[1].plot(k_list[: res_idx + 2], np.array(m_list[: res_idx + 2]) / 1e6)
        plt.xlabel("Curvature")
        plt.ylabel("Moment")
        plt.title("Moment-Curvature")
        fig.axes[1].set_xlim(0, 2.5e-5)
        fig.axes[1].set_ylim(0, 3700)
        plt.grid(True)

    return ax

## Generate Images

In [ ]:
for i in range(len(stress_res_list)):
    plot_stress_and_graph(
        i,
        stress_res_list,
        kappa_list[1:],
        moment_list[1:],
        ncols=2,
        filename=f"anim/{i:03d}.png",
        dpi=200,
    )

## Generate GIF

In [ ]:
import glob

from PIL import Image

# filepaths
fp_in = "anim/*.png"
fp_out = "anim.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
imgs = (Image.open(f) for f in sorted(glob.glob(fp_in)))
img = next(imgs)  # extract first image from iterator
img.save(
    fp=fp_out,
    format="GIF",
    append_images=imgs,
    save_all=True,
    duration=1000 / 24,
    loop=0,
)